#Web-Scraping

In [55]:
!pip install requests beautifulsoup4

##Libraries Necessary

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [8]:
headers = {
    'User-Agent': 'Mozilla/5.0'
}

In [9]:
# Step 1: Get links from homepage + section pages
base_url = 'https://www.filgoal.com'
start_url = f'{base_url}/'
response = requests.get(start_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [10]:
# Extract article links from homepage
article_links = set()
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if re.match(r'^/articles/\d+', href):
        article_links.add(base_url + href)

In [16]:
# Loop to generate the section URLs for the first 500 pages

for page_number in range(500):  # Loop from page 0 to 499
    sec_url = f'{base_url}/articles?page={page_number}'

    try:
        res = requests.get(sec_url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')

        # Extract article links from the current page
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if re.match(r'^/articles/\d+', href):  # Check if it's an article link
                article_links.add(base_url + href)

        print(f"Scraped page {page_number + 1} / 10")
        time.sleep(1)  # Add delay between requests to avoid hitting the server too fast

    except Exception as e:
        print(f"Error scraping page {page_number + 1}: {e}")
        time.sleep(2)  # Longer delay in case of failure to avoid further issues

Scraped page 1 / 10
Scraped page 2 / 10
Scraped page 3 / 10
Scraped page 4 / 10
Scraped page 5 / 10
Scraped page 6 / 10
Scraped page 7 / 10
Scraped page 8 / 10
Scraped page 9 / 10
Scraped page 10 / 10


##Scraping Each Article

In [20]:
# Scrape each article
articles = []
for i, link in enumerate(article_links):  # Scraping all article links
    try:
        res = requests.get(link, headers=headers)
        page = BeautifulSoup(res.content, 'html.parser')

        # Title
        title_tag = page.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "No title"

        # Body
        body_text = ""
        content_div = page.find('div', class_=re.compile(r'(articleBody|article-content|mainContent|newsDetails|story-content)'))
        if content_div:
            paragraphs = content_div.find_all('p')
        else:
            paragraphs = page.find_all('p')

        body_text = '\n'.join([p.get_text(strip=True) for p in paragraphs if len(p.get_text(strip=True)) > 50])

        if body_text:
            articles.append({
                'title': title,
                'body': body_text.strip()
            })

        print(f"Scraped ({i+1}/{len(article_links)}): {title}")
        time.sleep(1)

    except Exception as e:
        print(f"Error on {link}: {e}")

Scraped (1/34): سعودي في الجول
Scraped (2/34): الدوري المصري
Scraped (3/34): الكرة الأوروبية
Scraped (4/34): ميركاتو
Scraped (5/34): كرة يد
Scraped (6/34): كرة يد
Scraped (7/34): الكرة المصرية
Scraped (8/34): الدوري المصري
Scraped (9/34): الكرة المصرية
Scraped (10/34): الكرة الإفريقية
Scraped (11/34): كرة السلة الأمريكية
Scraped (12/34): الكرة المصرية
Scraped (13/34): الكرة الإفريقية
Scraped (14/34): كرة يد
Scraped (15/34): سعودي في الجول
Scraped (16/34): الدوري الإنجليزي
Scraped (17/34): الكرة المصرية
Scraped (18/34): الكرة الأوروبية
Scraped (19/34): الدوري الإنجليزي
Scraped (20/34): الدوري المصري
Scraped (21/34): الدوري المصري
Scraped (22/34): الكرة المصرية
Scraped (23/34): الكرة الإفريقية
Scraped (24/34): الدوري الإسباني
Scraped (25/34): ميركاتو
Scraped (26/34): الكرة الأوروبية
Scraped (27/34): الكرة الأوروبية
Scraped (28/34): سعودي في الجول
Scraped (29/34): الكرة الأوروبية
Scraped (30/34): الدوري المصري
Scraped (31/34): كرة طائرة
Scraped (32/34): الدوري المصري
Scraped (33/34): الكر

In [21]:
# Convert to DataFrame
df = pd.DataFrame(articles)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   34 non-null     object
 1   body    34 non-null     object
dtypes: object(2)
memory usage: 676.0+ bytes


In [33]:
df.head()

,title,body
0,سعودي في الجول,وافق 47 ناد سعودي على مشروع توثيق تاريخ الكرة ...
1,الدوري المصري,تلقى الأهلي عرضا رسميا من نادي فالنسيا الإسبان...
2,الكرة الأوروبية,عبر أوريلين تشواميني لاعب ريال مدريد عن سعادته...
3,ميركاتو,يسعى نادي الهلال لتعويض رحيل البرازيلي نيمار ع...
4,كرة يد,كشف ماركوس ماركيز مدرب حراس مرمى الزمالك لكرة ...


In [40]:
# Output the first title and body
if articles:
    print(f"Title: {articles[0]['title']}\n\nBody:\n{articles[0]['body']}")
else:
    print("No articles found.")

Title: سعودي في الجول

Body:
وافق 47 ناد سعودي على مشروع توثيق تاريخ الكرة السعودية، فيما أبدى نادي الشباب اعتراضه.
وغادر ممثل نادي الشباب الاجتماع الأخير لمشروع توثيق تاريخ الكرة السعودية وذلك قبل إصدار التقرير النهائي بنتائج المشروع.
وبحسب صحيفة الشرق الأوسط السعودية، فإن فارس العلياني ممثل نادي الشباب رفع البطاقة الحمراء اعتراضا على المصادقة على التقرير النهائي المتعلق بنتائج المشروع.
وقال العلياني عبر وسال إعلام سعودية: "أحسن الله عزاءنا في تاريخنا الرياضي، ما حدث هو عبث، والمتضرر الوحيد في هذا المشروع هو نادي الشباب".
وأضاف"نحن كُنا رابع الترتيب في سجل أبطال الدوري ثم أصبحنا خامسا. اتحاد القدم رمى موضوع التوثيق لدى الجمعية العمومية، ومع الأسف أندية ليس لها تاريخ رياضي تصوت وتقرر".
وستقوم الأندية بإعلان عدد بطولاتها وفقا لنتائج التقرير بعد المصادقة على التقرير النهائي لمشروع توثيق تاريخ كرة القدم السعودية، ، بعد ذلك سيتم الإعلان عنها في موقع الاتحاد السعودي لكرة القدم.
وأشارت التقارير إلى أن فريق عمل مشروع توثيق تاريخ كرة القدم السعودية بين في تقريره أن عدد البطولات المرصودة 3910 ب

##Saving Data into CSV

In [29]:
# Save collected data to CSV file
df.to_csv("filgoal_articles.csv", index=False, encoding='utf-8-sig')

#Lang-Chain

In [73]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import faiss
import torch
import numpy as np

##Data Preprocessing

In [42]:
# Data Loading
file_path_csv = '/content/filgoal_articles.csv'
data = pd.read_csv(file_path_csv)

In [43]:
# Combine title and body for better context
data['full_text'] = data['title'] + "\n" + data['body']

In [78]:
# Data Splitting with optimal chunk size and overlap
chunk_size = 700  # Optimal chunk size to maintain context (adjust as needed)
chunk_overlap = 100  # Overlap between chunks to maintain context

In [79]:
# Create Document objects
documents = [Document(page_content=text) for text in data['full_text'].tolist()]

In [80]:
# Initialize RecursiveCharacterTextSplitter with the optimal chunk size and overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [81]:
# Split the text into chunks
split_documents = splitter.split_documents(documents)

In [82]:
# Extract the text to be embedded
texts = [doc.page_content for doc in split_documents]

In [83]:
# Generate embeddings using AraBERT
# Load AraBERT model and tokenizer for Arabic embeddings
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv02")

##araBERT Model

In [84]:
# Function to get embeddings for a list of texts
def get_arabert_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()  # Taking the mean of token embeddings
    return embeddings

In [85]:
# Generate embeddings for the text chunks using AraBERT
embeddings = get_arabert_embeddings(texts)

# Convert embeddings to numpy array for FAISS compatibility
embeddings_np = np.array(embeddings).astype('float32')

In [86]:
# Create FAISS Index
index = faiss.IndexFlatL2(embeddings_np.shape[1])  # L2 distance for similarity search
index.add(embeddings_np)  # Add embeddings to FAISS index

##Question Answering

In [87]:
# Question Answering using FAISS Index

def answer_query(query, model, tokenizer, index, top_k=2):
    # Convert the query into an embedding using AraBERT
    query_inputs = tokenizer([query], padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        query_outputs = model(**query_inputs)
    query_embedding = query_outputs.last_hidden_state.mean(dim=1).numpy().astype('float32')

    # Perform similarity search (get the top_k most similar chunks)
    D, I = index.search(query_embedding, top_k)  # D is distances, I is indices of nearest neighbors

    # Return the most similar text chunks as the answer
    answers = [texts[i] for i in I[0]]
    return answers

##Testing Our Model

In [88]:
# Example Test
query_arabic = "ما هي أحدث أخبار الانتقالات في كرة القدم؟"  # Example Arabic query
answers_arabic = answer_query(query_arabic, model, tokenizer, index)

# Answers in Arabic
for answer in answers_arabic:
    print(answer)
    print("=" * 100)

ميركاتو
ذكرت شبكة سكاي سبورت ألمانيا أن نادي باير ليفركوزن يتواصل مع مانشستر سيتي بشكل مباشر من أجل ضم الجناح جيمس ماكاتي.
وأوضح التقرير أن الناديان يدرسان كل الحلول الممكنة لإتمام الصفقة.
وأضاف التقرير أن هناك أندية أخرى من الدوري الإنجليزي والدوري الإيطالي أبدت اهتمامها بضم اللاعب.
لم يتوصل ماكاتي إلى اتفاق شفهي مع أي نادٍ حتى الآن، حسب التقرير.
ولفت التقرير إلى أن بيب جوارديولا مدرب سيتي يرغب في الاحتفاظ بماكاتي، لكن اللاعب يريد الرحيل لرغبته في لعب دقائق أكثر.
الجناح الشاب يشارك في بعض المباريات مع بيب جوارديولا هذا الموسم، فقد لعب 24 مباراة وسجل 7 أهداف.
وحاول ليفركوزن ضم اللاعب في يناير الماضي على سبيل الإعارة مع خيار الشراء، لكن الصفقة لم تتم.
ولكن ماذا لو كنت تملك لجنة التخطيط في وقت مبكر أو ما يساويها في عملها كمدير رياضي أو لجنة كرة، مهما كان المسمى؟ بالتأكيد بيع لاعب بهذا المبلغ الخيالي واستقطاب محترفين في قيمة أشرف بنشرقي وفرجاني ساسي وليس كونراد ميشالاك البولندي الكارثي.
بالتالي عندما كتبت أن المسألة ليس أموالا فقط، فأقصدها بالضبط، لأن لا توجد قاعدة بمن يصرف أكثر يفوز.
هذه

In [90]:
# Example Test
query_arabic = "ما هي آخر أخبار الدوري المصري؟"  # Example Arabic query
answers_arabic = answer_query(query_arabic, model, tokenizer, index)

# Answers in Arabic
for answer in answers_arabic:
    print(answer)
    print("=" * 100)

الدوري المصري
أوضح مصدر مقرب من أحمد سيد "زيزو" لاعب الزمالك أنه لا يمانع استكمال الموسم مع الأبيض.
وقال المصدرلـFilGoal.com:"زيزو يتدرب بشكل مستمر في الوقت الحالي منتظرا تحديد موقفه مع الزمالك وهو لا يرفض استكمال الموسم مع الزمالك ولكن بعد الحصول على الضمانات المطلوبة".
وأضاف "في حالة عودته للتدريبات فهو جاهز لذلك وللمشاركة بشكل طبيعي في المباريات مع الزمالك، وإذا لم تحل الأزمة فسيكون جاهزا للمشاركة مع الفريق الجديد الذي سينتقل له في الموسم المقبل".
وأتم تصريحاته "زيزو لاعب محترف ويعرف كيف يحافظ على جاهزيته ويتدرب مع مدرب خاص بشكل يومي".
كشف مصدر من نادي الزمالك في وقت سابقلـFilGoal.comعن حقيقة المستحقات المالية المتأخرة للاعب أحمد سيد زيزو، والتي يقدرها اللاعب بقرابة 73 مليون جنيه.
ولكن في عهد الحاج -بصفته الوظيفية لحين أداء الفريضة- طه عزت، صار أمرا طبيعيا أن تعلن رابطة الأندية يوم 8 مارس، أن الأهلي والزمالك سيلتقيان يوم 13 مارس، في وجود لائحة تلزم بالإبلاغ بطلب الحكام الأجانب قبلها بـ 14 يوما، على فرض استحالة الإعلان عن إقامة مباراة "في بطولة دوري" قبل إقامتها بمدة أقل من هذه.
بطبي

In [93]:
# Example Test
query_arabic = "ما هي أخبار الدوري السعودي؟"  # Example Arabic query
answers_arabic = answer_query(query_arabic, model, tokenizer, index)

# Answers in Arabic
for answer in answers_arabic:
    print(answer)
    print("=" * 100)

سعودي في الجول
وافق 47 ناد سعودي على مشروع توثيق تاريخ الكرة السعودية، فيما أبدى نادي الشباب اعتراضه.
وغادر ممثل نادي الشباب الاجتماع الأخير لمشروع توثيق تاريخ الكرة السعودية وذلك قبل إصدار التقرير النهائي بنتائج المشروع.
وبحسب صحيفة الشرق الأوسط السعودية، فإن فارس العلياني ممثل نادي الشباب رفع البطاقة الحمراء اعتراضا على المصادقة على التقرير النهائي المتعلق بنتائج المشروع.
وقال العلياني عبر وسال إعلام سعودية: "أحسن الله عزاءنا في تاريخنا الرياضي، ما حدث هو عبث، والمتضرر الوحيد في هذا المشروع هو نادي الشباب".
وأضاف"نحن كُنا رابع الترتيب في سجل أبطال الدوري ثم أصبحنا خامسا. اتحاد القدم رمى موضوع التوثيق لدى الجمعية العمومية، ومع الأسف أندية ليس لها تاريخ رياضي تصوت وتقرر".
ولكن في عهد الحاج -بصفته الوظيفية لحين أداء الفريضة- طه عزت، صار أمرا طبيعيا أن تعلن رابطة الأندية يوم 8 مارس، أن الأهلي والزمالك سيلتقيان يوم 13 مارس، في وجود لائحة تلزم بالإبلاغ بطلب الحكام الأجانب قبلها بـ 14 يوما، على فرض استحالة الإعلان عن إقامة مباراة "في بطولة دوري" قبل إقامتها بمدة أقل من هذه.
بطبيعة الحال، نع